# 창폐업 데이터 확인

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# # 한글폰트 사용 in colab
# %matplotlib inline  

# import matplotlib as mpl 
# import matplotlib.pyplot as plt 
# import matplotlib.font_manager as fm  

# !apt-get update -qq
# !apt-get install fonts-nanum* -qq

# path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
# font_name = fm.FontProperties(fname=path, size=10).get_name()
# print(font_name)
# plt.rc('font', family=font_name)

# fm._rebuild()
# mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import os
import time
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [ ]:
os.listdir("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/")

In [ ]:
region = pd.read_csv("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/StartupOrClosing_Region_ver2.csv")
shop = pd.read_csv("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/StartupOrClosing_Shop_ver2.csv")
display(region.head())
display(shop.head())

In [ ]:
region['INDUTY_NM'].unique()

In [ ]:
# 날짜가 폐업 날짜가 아닌 데이터 생성날짜
# 어떻게 활용할 수 있을지? ==> 업종별로 파악하는 수밖에 없음
# 딱히 일자를 맞춰서 쓰기 보단, 총합을 계산해야 할 듯 하다.
# 1월 20일 전은 데이터 생성 시기로 따져봐도 코로나 전으로 판단
# 1월 20일 후부터 코로나 영향권으로 볼 지에 대한 의문
# 배달 호출데이터는 3월 31일까지 이므로, 그 후 데이터 사용?


# 날짜에 대해선 팀원과 논의 후 결정

print(region['DATE'].unique())
print(shop['DATE'].unique())

In [ ]:
# 위도 경도로 특정 구에 얼마나 폐업했는지는 파악 가능
# 다만 지역을 서울, 경기로 한정 ==> 배달데이터가 서울 경기
sudo = region[(region['BRTC_NM']=='서울') | (region['BRTC_NM']=='경기')]
sudo

In [ ]:
# 날짜 상관없이 창업 폐업만 알아볼까?
sudo['DIVISION'].value_counts() # 폐업이 압도적으로 많다. ==> 나중에 시간순으로도 보면 좋을 듯

In [ ]:
# 서울 내 특정 구 확인을 위한 전처리
# replace가 안먹히네
# 그냥 슬라이싱으로 처리

for x in tqdm(range(len(sudo))) :
  sudo['LA_DCMLPOINT_VALUE'].iloc[x] = sudo['LA_DCMLPOINT_VALUE'].iloc[x][:-3]
  sudo['LO_DCMLPOINT_VALUE'].iloc[x] = sudo['LO_DCMLPOINT_VALUE'].iloc[x][:-3]

sudo.head()

In [ ]:
# 아이디 만들기
sudo['ID'] = sudo['LA_DCMLPOINT_VALUE'] + str("::") + sudo['LO_DCMLPOINT_VALUE']
sudo.head()

In [ ]:
sudo.groupby(by='ID', as_index=False).size() # 알아보기 위해서는 점을 찍고 마커클러스터로 수를 확인하는 것이 가장 효율적일 듯 하다.
## 아니면 뒤에 한자리씩 자르고 size 수를 늘려도 될 것 같긴 하다.
## 클러스터와 공시지가를 합쳐서 시각화하는 것도 가능하다.


In [ ]:
sudo.groupby(by='ID', as_index=False).size()['size'].max()

In [ ]:
######################################################### 번외편 주문지역 인구특성

In [ ]:
pop = pd.read_csv("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/popl.csv")

In [ ]:
display(pop.head())

In [ ]:
################################ 시간대별 서울 창폐업 비교
seoul_region1 = region[region['BRTC_NM']=='서울']
seoul_region1['DATE'] = pd.to_datetime(seoul_region1['DATE'], format='%Y%m%d')
seoul_region1 = seoul_region1[seoul_region1['DATE'] < '2020-04-01']
seoul_region1['DATE'] = seoul_region1['DATE'].astype(str)
seoul_region1.head()

In [ ]:
plt.figure(figsize=(30,10))

sns.countplot(x='DATE', hue='DIVISION', data=seoul_region1)
plt.title("날짜별 창폐업 비교")
plt.xticks(rotation=90)
plt.show()
# 후반부에 엄청 폐업 날짜가 몰려있는 날이 존재 // 해당 날짜에 거의 모든 데이터가 있는 듯 하다.

In [ ]:
##### 원 데이터 비교
seoul_region = region[region['BRTC_NM']=='서울']
plt.figure(figsize=(30,10))

sns.countplot(x='DATE', hue='DIVISION', data=seoul_region)
plt.title("날짜별 창폐업 비교")
plt.xticks(rotation=90)
plt.show()

# 2020년 5월 29일에 도대체 무슨 일이...